# The fast spectral method for 2D

In [4]:
import numpy as np
from math import pi
from absl import logging
import time

logging.set_verbosity("info")


In [5]:
from kipack.collision.inelastic import FSInelasticVHSCollision
from kipack.collision.vmesh import SpectralMesh
from config_2d import get_config


In [6]:
def bkw_f(v):
    t = 0.5
    K = 1 - 0.5 * np.exp(-t / 8)
    v_norm = (v ** 2)[:, None] + v ** 2
    return (
        1
        / (2 * pi * K ** 2)
        * np.exp(-0.5 * v_norm / K)
        * (2 * K - 1 + 0.5 * v_norm * (1 - K) / K)
    )


def ext_Q(v):
    t = 0.5
    K = 1 - np.exp(-t / 8) / 2
    dK = np.exp(-t / 8) / 16
    v_norm = (v ** 2)[:, None] + v ** 2
    df = (-2 / K + v_norm / (2 * K ** 2)) * bkw_f(v) + 1 / (
        2 * pi * K ** 2
    ) * np.exp(-v_norm / (2 * K)) * (2 - v_norm / (2 * K ** 2))
    return df * dK


In [7]:
cfg = get_config()
vmesh = SpectralMesh(cfg)
coll = FSInelasticVHSCollision(cfg, vmesh)


INFO:absl:2 dimensional collision model.
INFO:absl:Number of velocity cells: 64.
INFO:absl:Velocity domain: [-7.724873734152916, 7.724873734152916].
INFO:absl:e: 1.0
INFO:absl:Collision model precomputation finished!


In [9]:
t_0 = time.time()
Q = coll(bkw_f(vmesh.center), device="gpu")
dt = time.time() - t_0

print(
    f"Runtime: {1000* dt:.2f}ms, error: {np.max(np.abs(Q - ext_Q(vmesh.center)))}"
)


Runtime: 5.40ms, error: 1.7104168583484674e-11
